In [7]:
import pandas as pd 

train = pd.read_csv("../data/train.csv", parse_dates=["date"])

train.head()



,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [ ]:
# Pick a store and Family
focus_store  = 1            
focus_family = "GROCERY I"

df = train[(train["store_nbr"] == focus_store) &
           (train["family"]    == focus_family)].copy()

print(df.shape)    
df.head()          


(1684, 6)


,id,date,store_nbr,family,sales,onpromotion
12,12,2013-01-01,1,GROCERY I,0.0,0
1794,1794,2013-01-02,1,GROCERY I,2652.0,0
3576,3576,2013-01-03,1,GROCERY I,2121.0,0
5358,5358,2013-01-04,1,GROCERY I,2056.0,0
7140,7140,2013-01-05,1,GROCERY I,2216.0,0
